In [7]:
# source( "~/labo/src/lightgbm/z623_lightgbm_binaria_BO.r" )
# Este script esta pensado para correr en Google Cloud
#   8 vCPU
#  32 GB memoria RAM
# 256 GB espacio en disco

# se entrena con POS =  { BAJA+1, BAJA+2 }
# Optimizacion Bayesiana de hiperparametros de  lightgbm, con el metodo TRADICIONAL de los hiperparametros originales de lightgbm
# 5-fold cross validation
# la probabilidad de corte es un hiperparametro

#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

require("data.table")
require("rlist")

require("lightgbm")

#paquetes necesarios para la Bayesian Optimization
require("DiceKriging")
require("mlrMBO")

options(error = function() { 
  traceback(20); 
  options(error = NULL); 
  stop("exiting after script error") 
})


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2426055,129.6,4352334,232.5,4352334,232.5
Vcells,71261360,543.7,203290476,1551.0,203290127,1551.0


In [8]:
#Aqui se debe poner la carpeta de la computadora local
setwd("~/buckets/b1/")   #Establezco el Working Directory

#setwd("C:/_MCD/Labo1/code")  #Establezco el Working Directory
#kdataset       <- "./datasets/competencia2_2022.csv.gz"
kdataset       <- "./datasets/competencia1_2022_FE_DC2.csv"

In [9]:
kBO_iter  <- 100   #cantidad de iteraciones de la Optimizacion Bayesiana

# ATENCION  si NO se quiere utilizar  undersampling  se debe  usar  kundersampling <- 1.0
kundersampling  <- 1.0   # un undersampling de 0.1  toma solo el 10% de los CONTINUA

prob_min  <- 0.5/( 1 + kundersampling*39)
prob_max  <- pmin( 1.0, 4/( 1 + kundersampling*39) )

#Aqui se cargan los hiperparametros
hs <- makeParamSet( 
         makeNumericParam("learning_rate",    lower=  0.01   , upper=    0.3),
         makeNumericParam("feature_fraction", lower=  0.2    , upper=    1.0),
         makeIntegerParam("min_data_in_leaf", lower=  0      , upper= 8000),
         makeIntegerParam("num_leaves",       lower= 16L     , upper= 1024L),
         makeNumericParam("prob_corte",       lower= prob_min, upper= prob_max  )  #esto sera visto en clase en gran detalle
        )

ksemilla_azar  <- 102191  #Aqui poner la propia semilla
kexperimento   <- "HT6230"
ktraining      <- c( 202101 )   #periodos en donde entreno

kPOS_ganancia  <- 78000
kNEG_ganancia  <- -2000

In [10]:
loguear  <- function( reg, arch=NA, folder="./exp/", ext=".txt", verbose=TRUE )
{
  archivo  <- arch
  if( is.na(arch) )  archivo  <- paste0(  folder, substitute( reg), ext )

  if( !file.exists( archivo ) )  #Escribo los titulos
  {
    linea  <- paste0( "fecha\t", 
                      paste( list.names(reg), collapse="\t" ), "\n" )

    cat( linea, file=archivo )
  }

  linea  <- paste0( format(Sys.time(), "%Y%m%d %H%M%S"),  "\t",     #la fecha y hora
                    gsub( ", ", "\t", toString( reg ) ),  "\n" )

  cat( linea, file=archivo, append=TRUE )  #grabo al archivo

  if( verbose )  cat( linea )   #imprimo por pantalla
}
#------------------------------------------------------------------------------
#esta funcion calcula internamente la ganancia de la prediccion probs

fganancia_logistic_lightgbm   <- function( probs, datos) 
{
  vlabels  <- get_field(datos, "label")
  vpesos   <- get_field(datos, "weight")

  gan  <- sum( (probs > PROB_CORTE  ) *
               ifelse( vpesos == 1.0000002, kPOS_ganancia, 
                       ifelse( vpesos == 1.0000001, kNEG_ganancia, kNEG_ganancia / kundersampling ) ) )


  return( list( "name"= "ganancia", 
                "value"=  gan,
                "higher_better"= TRUE ) )
}
#------------------------------------------------------------------------------
#esta funcion solo puede recibir los parametros que se estan optimizando
#el resto de los parametros se pasan como variables globales, la semilla del mal ...

EstimarGanancia_lightgbm  <- function( x )
{
  gc()  #libero memoria

  #llevo el registro de la iteracion por la que voy
  GLOBAL_iteracion  <<- GLOBAL_iteracion + 1

  PROB_CORTE <<- x$prob_corte   #asigno la variable global

  kfolds  <- 5   # cantidad de folds para cross validation

  param_basicos  <- list( objective= "binary",
                          metric= "custom",
                          first_metric_only= TRUE,
                          boost_from_average= TRUE,
                          feature_pre_filter= FALSE,
                          verbosity= -100,
                          max_depth=  -1,         # -1 significa no limitar,  por ahora lo dejo fijo
                          min_gain_to_split= 0.0, #por ahora, lo dejo fijo
                          lambda_l1= 0.0,         #por ahora, lo dejo fijo
                          lambda_l2= 0.0,         #por ahora, lo dejo fijo
                          max_bin= 31,            #por ahora, lo dejo fijo
                          num_iterations= 9999,   #un numero muy grande, lo limita early_stopping_rounds
                          force_row_wise= TRUE,   #para que los alumnos no se atemoricen con tantos warning
                          seed= ksemilla_azar
                        )

  #el parametro discolo, que depende de otro
  param_variable  <- list(  early_stopping_rounds= as.integer(50 + 5/x$learning_rate) )

  param_completo  <- c( param_basicos, param_variable, x )

  set.seed( ksemilla_azar )
  modelocv  <- lgb.cv( data= dtrain,
                       eval= fganancia_logistic_lightgbm,
                       stratified= TRUE, #sobre el cross validation
                       nfold= kfolds,    #folds del cross validation
                       param= param_completo,
                       verbose= -100
                      )

  #obtengo la ganancia
  ganancia  <- unlist(modelocv$record_evals$valid$ganancia$eval)[ modelocv$best_iter ]

  ganancia_normalizada  <-  ganancia* kfolds     #normailizo la ganancia

  #el lenguaje R permite asignarle ATRIBUTOS a cualquier variable
  attr(ganancia_normalizada ,"extras" )  <- list("num_iterations"= modelocv$best_iter)  #esta es la forma de devolver un parametro extra

  param_completo$num_iterations <- modelocv$best_iter  #asigno el mejor num_iterations
  param_completo["early_stopping_rounds"]  <- NULL     #elimino de la lista el componente  "early_stopping_rounds"

  #logueo 
  xx  <- param_completo
  xx$ganancia  <- ganancia_normalizada   #le agrego la ganancia
  xx$iteracion <- GLOBAL_iteracion
  loguear( xx, arch= klog )

  return( ganancia_normalizada )
}

In [11]:
#cargo el dataset donde voy a entrenar el modelo
dataset  <- fread( kdataset)

#creo la carpeta donde va el experimento
# HT  representa  Hiperparameter Tuning

#en estos archivos quedan los resultados
kbayesiana  <- paste0( kexperimento, ".RDATA" )
klog        <- paste0( kexperimento, ".txt" )


GLOBAL_iteracion  <- 0   #inicializo la variable global

#si ya existe el archivo log, traigo hasta donde llegue
if( file.exists(klog) )
{
  tabla_log  <- fread( klog )
  GLOBAL_iteracion  <- nrow( tabla_log )
}



#paso la clase a binaria que tome valores {0,1}  enteros
dataset[ foto_mes %in% ktraining, clase01 := ifelse( clase_ternaria=="CONTINUA", 0L, 1L) ]


#los campos que se van a utilizar
campos_buenos  <- setdiff( colnames(dataset), c("clase_ternaria","clase01", "azar", "training" ) )

set.seed( ksemilla_azar )
dataset[  , azar := runif( nrow( dataset ) ) ]
dataset[  , training := 0L ]
dataset[ foto_mes %in% ktraining & ( azar <= kundersampling | clase_ternaria %in% c( "BAJA+1", "BAJA+2" ) ), training := 1L ]

#dejo los datos en el formato que necesita LightGBM
dtrain  <- lgb.Dataset( data= data.matrix(  dataset[ training == 1L, campos_buenos, with=FALSE]),
                        label= dataset[ training == 1L, clase01 ],
                        weight=  dataset[ training == 1L, ifelse( clase_ternaria=="BAJA+2", 1.0000002, ifelse( clase_ternaria=="BAJA+1",  1.0000001, 1.0) )],
                        free_raw_data= FALSE  )



#Aqui comienza la configuracion de la Bayesian Optimization
funcion_optimizar  <- EstimarGanancia_lightgbm   #la funcion que voy a maximizar

configureMlr( show.learner.output= FALSE)

#configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
#por favor, no desesperarse por lo complejo
obj.fun  <- makeSingleObjectiveFunction(
              fn=       funcion_optimizar, #la funcion que voy a maximizar
              minimize= FALSE,   #estoy Maximizando la ganancia
              noisy=    TRUE,
              par.set=  hs,     #definido al comienzo del programa
              has.simple.signature = FALSE   #paso los parametros en una lista
             )

ctrl  <- makeMBOControl( save.on.disk.at.time= 600,  save.file.path= kbayesiana)  #se graba cada 600 segundos
ctrl  <- setMBOControlTermination(ctrl, iters= kBO_iter )   #cantidad de iteraciones
ctrl  <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI() )

#establezco la funcion que busca el maximo
surr.km  <- makeLearner("regr.km", predict.type= "se", covtype= "matern3_2", control= list(trace= TRUE))




In [12]:

#inicio la optimizacion bayesiana
if( !file.exists( kbayesiana ) ) {
  run  <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  run  <- mboContinue( kbayesiana )   #retomo en caso que ya exista
}


Computing y column(s) for design. Not provided.



[LightGBM] [Warning] Unknown parameter: prob_corte
[LightGBM] [Warning] verbosity is set=-100, verbose=-100 will be ignored. Current value: verbosity=-100
[LightGBM] [Warning] Unknown parameter: prob_corte
[LightGBM] [Warning] verbosity is set=-100, verbose=-100 will be ignored. Current value: verbosity=-100
[LightGBM] [Warning] Unknown parameter: prob_corte
[LightGBM] [Warning] verbosity is set=-100, verbose=-100 will be ignored. Current value: verbosity=-100
[LightGBM] [Warning] Unknown parameter: prob_corte
[LightGBM] [Warning] verbosity is set=-100, verbose=-100 will be ignored. Current value: verbosity=-100
[LightGBM] [Warning] Unknown parameter: prob_corte
[LightGBM] [Warning] verbosity is set=-100, verbose=-100 will be ignored. Current value: verbosity=-100
[LightGBM] [Warning] Unknown parameter: prob_corte
[LightGBM] [Warning] verbosity is set=-100, verbose=-100 will be ignored. Current value: verbosity=-100
[LightGBM] [Info] [binary:BoostFromScore]: pavg=1.000000 -> initscore=

[mbo] 0: learning_rate=0.0573; feature_fraction=0.879; min_data_in_leaf=2924; num_leaves=737; prob_corte=0.0446 : y = -3.26e+08 : 4.7 secs : initdesign

[mbo] 0: learning_rate=0.253; feature_fraction=0.679; min_data_in_leaf=4805; num_leaves=471; prob_corte=0.0664 : y = -3.26e+08 : 2.1 secs : initdesign

[mbo] 0: learning_rate=0.151; feature_fraction=0.696; min_data_in_leaf=6093; num_leaves=451; prob_corte=0.0586 : y = -3.26e+08 : 2.5 secs : initdesign

[mbo] 0: learning_rate=0.203; feature_fraction=0.253; min_data_in_leaf=3828; num_leaves=163; prob_corte=0.0291 : y = -3.26e+08 : 2.3 secs : initdesign

[mbo] 0: learning_rate=0.165; feature_fraction=0.918; min_data_in_leaf=3286; num_leaves=671; prob_corte=0.0626 : y = -3.26e+08 : 3.1 secs : initdesign

[mbo] 0: learning_rate=0.121; feature_fraction=0.741; min_data_in_leaf=7567; num_leaves=66; prob_corte=0.0826 : y = -3.26e+08 : 2.3 secs : initdesign

[mbo] 0: learning_rate=0.226; feature_fraction=0.484; min_data_in_leaf=4537; num_leaves=

[LightGBM] [Warning] Unknown parameter: prob_corte
[LightGBM] [Warning] verbosity is set=-100, verbose=-100 will be ignored. Current value: verbosity=-100
[LightGBM] [Warning] Unknown parameter: prob_corte
[LightGBM] [Warning] verbosity is set=-100, verbose=-100 will be ignored. Current value: verbosity=-100
[LightGBM] [Warning] Unknown parameter: prob_corte
[LightGBM] [Warning] verbosity is set=-100, verbose=-100 will be ignored. Current value: verbosity=-100
[LightGBM] [Warning] Unknown parameter: prob_corte
[LightGBM] [Warning] verbosity is set=-100, verbose=-100 will be ignored. Current value: verbosity=-100
[LightGBM] [Warning] Unknown parameter: prob_corte
[LightGBM] [Warning] verbosity is set=-100, verbose=-100 will be ignored. Current value: verbosity=-100
[LightGBM] [Warning] Unknown parameter: prob_corte
[LightGBM] [Warning] verbosity is set=-100, verbose=-100 will be ignored. Current value: verbosity=-100
[LightGBM] [Info] [binary:BoostFromScore]: pavg=1.000000 -> initscore=

[mbo] 1: learning_rate=0.241; feature_fraction=0.213; min_data_in_leaf=3771; num_leaves=75; prob_corte=0.0796 : y = -3.26e+08 : 2.5 secs : infill_ei



[LightGBM] [Warning] Unknown parameter: prob_corte
[LightGBM] [Warning] verbosity is set=-100, verbose=-100 will be ignored. Current value: verbosity=-100
[LightGBM] [Warning] Unknown parameter: prob_corte
[LightGBM] [Warning] verbosity is set=-100, verbose=-100 will be ignored. Current value: verbosity=-100
[LightGBM] [Warning] Unknown parameter: prob_corte
[LightGBM] [Warning] verbosity is set=-100, verbose=-100 will be ignored. Current value: verbosity=-100
[LightGBM] [Warning] Unknown parameter: prob_corte
[LightGBM] [Warning] verbosity is set=-100, verbose=-100 will be ignored. Current value: verbosity=-100
[LightGBM] [Warning] Unknown parameter: prob_corte
[LightGBM] [Warning] verbosity is set=-100, verbose=-100 will be ignored. Current value: verbosity=-100
[LightGBM] [Warning] Unknown parameter: prob_corte
[LightGBM] [Warning] verbosity is set=-100, verbose=-100 will be ignored. Current value: verbosity=-100
[LightGBM] [Info] [binary:BoostFromScore]: pavg=1.000000 -> initscore=

[mbo] 2: learning_rate=0.241; feature_fraction=0.213; min_data_in_leaf=3771; num_leaves=75; prob_corte=0.0796 : y = -3.26e+08 : 2.5 secs : infill_ei



ERROR: Error in chol.default(R): la submatriz de orden 22 no es definida positiva


In [ ]:
quit( save="no" )